### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2022, 12, 19)

### Process after last business day, yesterday must be last business day

In [2]:
# specify the number of business days
num_days = 1
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(num_days)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2022-12-19
yesterday: 2022-12-16 00:00:00


In [3]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 7)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2022-12-09
yesterday: 2022-12-16


### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [4]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [5]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [6]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,SINGER,2022-12-16,28.00


In [7]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'SINGER'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,446,SINGER,SET100 / SETWB,27.50,59.25,26.25,23.69,1.50,822.27,"22,612.33",202.07,2.14,446,2017-07-23 07:25:25.725583,2022-12-17 16:01:18.197165


In [8]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "SINGER" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,SINGER,2022-12-16,27.50,28.50,26.25,"14,868,777",28.50
1,SINGER,2022-12-15,28.50,30.75,28.25,"12,997,092",30.75
2,SINGER,2022-12-14,31.00,31.50,30.75,"2,774,304",30.75
3,SINGER,2022-12-13,30.75,31.75,30.25,"4,226,734",31.50
4,SINGER,2022-12-09,31.50,32.25,31.50,"2,522,595",32.00


In [9]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-12-16' AND name IN ('ASP', 'BCH', 'CPNREIT', 'DCC', 'DIF', 'DOHOME', 'GVREIT', 'IVL', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SCCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART')
ORDER BY name, date


26

In [10]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-12-09' AND name IN ('ASP', 'BCH', 'CPNREIT', 'DCC', 'DIF', 'DOHOME', 'GVREIT', 'IVL', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SCCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART') 
ORDER BY name


26

In [11]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(26, 3)

In [12]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [13]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [14]:
trend

,name,price_d,price_w,percent
0,ASP,2.96,2.98,-0.67
1,BCH,19.80,19.80,0.00
2,CPNREIT,19.50,19.00,2.63
3,DCC,2.78,2.80,-0.71
4,DIF,13.10,13.20,-0.76
5,DOHOME,14.00,14.20,-1.41
6,GVREIT,8.95,8.95,0.00
7,IVL,40.00,40.75,-1.84
8,JASIF,8.05,8.00,0.63
9,KCE,47.25,49.75,-5.03


In [15]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [16]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
19,SINGER,27.50,31.50,-12.70%,Worse
21,SYNEX,15.90,17.70,-10.17%,Worse
9,KCE,47.25,49.75,-5.03%,Worse
7,IVL,40.00,40.75,-1.84%,Worse
5,DOHOME,14.00,14.20,-1.41%,Worse
4,DIF,13.10,13.20,-0.76%,Worse
3,DCC,2.78,2.80,-0.71%,Worse
0,ASP,2.96,2.98,-0.67%,Worse
16,SCC,333.00,334.00,-0.30%,Worse
18,SENA,3.96,3.96,0.00%,No Change


In [17]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,42.31%
Worse,34.62%
No Change,23.08%


In [18]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(253, 8)

In [19]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [20]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(26, 13)

In [21]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
9,KCE,47.25,49.75,-5.03%,Worse,SET50
7,IVL,40.00,40.75,-1.84%,Worse,SET50
16,SCC,333.00,334.00,-0.30%,Worse,SET50
14,PTTGC,46.75,46.00,1.63%,Better,SET50


In [22]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,75.00%
Better,25.00%


In [23]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
19,SINGER,27.50,31.50,-12.70%,Worse,SET100
21,SYNEX,15.90,17.70,-10.17%,Worse,SET100
5,DOHOME,14.00,14.20,-1.41%,Worse,SET100
1,BCH,19.80,19.80,0.00%,No Change,SET100
13,ORI,10.90,10.90,0.00%,No Change,SET100
20,STA,19.10,18.80,1.60%,Better,SET100
15,RCL,30.00,29.00,3.45%,Better,SET100


In [24]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,42.86%
No Change,28.57%
Better,28.57%


In [25]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
4,DIF,13.10,13.20,-0.76%,Worse,SET999
3,DCC,2.78,2.80,-0.71%,Worse,SET999
0,ASP,2.96,2.98,-0.67%,Worse,SET999
6,GVREIT,8.95,8.95,0.00%,No Change,SET999
11,MCS,8.95,8.95,0.00%,No Change,SET999
12,NER,5.95,5.95,0.00%,No Change,SET999
18,SENA,3.96,3.96,0.00%,No Change,SET999
17,SCCC,149.00,148.50,0.34%,Better,SET999
8,JASIF,8.05,8.00,0.63%,Better,SET999
25,WHART,10.20,10.10,0.99%,Better,SET999


In [26]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,53.33%
No Change,26.67%
Worse,20.00%


In [27]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt


In [28]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
